<a href="https://colab.research.google.com/github/bhuvana-ak/uplimit_projects/blob/main/Fine_tune_Llama_3.2_1B_with_ORPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U git+https://github.com/huggingface/transformers.git -qq
!pip install -U git+https://github.com/huggingface/trl.git -qq
!pip install -qqq -U datasets accelerate peft bitsandbytes wandb --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00


In [2]:
import gc
import os

import torch
import wandb
from datasets import load_dataset
from google.colab import userdata
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import ORPOConfig, ORPOTrainer, setup_chat_format

# Model
base_model = "meta-llama/Llama-3.2-1B"
new_model = "OrpoLlama-3.2-1B-V1"

# Defined in the secrets tab in Google Colab
wb_token = userdata.get('wb_token')
wandb.login(key=wb_token)

# # Set torch dtype and attention implementation
# if torch.cuda.get_device_capability()[0] >= 8:
#     !pip install -qqq flash-attn
#     torch_dtype = torch.bfloat16
#     attn_implementation = "flash_attention_2"
# else:
#     torch_dtype = torch.float16
#     attn_implementation = "eager"
torch_dtype = torch.float16
#attn_implementation = "eager"

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [4]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [5]:
model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [10]:
# Define max length
max_length = 2048  # or whatever your model's maximum is
dataset_name = "mlabonne/orpo-dpo-mix-40k"
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=42).select(range(1000)) # Only use 1000 samples for quick demo

def preprocess_dataset(example):
    tokenizer.truncation_side = "left"  # or "right", depending on your preference

    prompt = tokenizer.encode(example['prompt'], truncation=True, max_length=max_length)
    chosen_content = example['chosen'][0]['content'] if example['chosen'] else ""
    rejected_content = example['rejected'][0]['content'] if example['rejected'] else ""

    chosen = tokenizer.encode(chosen_content, truncation=True, max_length=max_length)
    rejected = tokenizer.encode(rejected_content, truncation=True, max_length=max_length)

    return {
        "prompt": tokenizer.decode(prompt),
        "chosen": tokenizer.decode(chosen),
        "rejected": tokenizer.decode(rejected)
    }

# Apply the preprocessing to your dataset
preprocessed_dataset = dataset.map(preprocess_dataset)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
# dataset_name = "mlabonne/orpo-dpo-mix-40k"
# dataset = load_dataset(dataset_name, split="all")
# dataset = dataset.shuffle(seed=42).select(range(1000)) # Only use 1000 samples for quick demo

# def format_chat_template(row):
#     row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
#     row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
#     return row

# dataset = dataset.map(
#     format_chat_template,
#     num_proc= os.cpu_count(),
# )
dataset = dataset.train_test_split(test_size=0.01)

In [28]:
dataset.column_names

{'train': ['source', 'chosen', 'rejected', 'prompt', 'question'],
 'test': ['source', 'chosen', 'rejected', 'prompt', 'question']}

In [16]:
orpo_args = ORPOConfig(
    learning_rate=8e-6,
    lr_scheduler_type="linear",
    max_length=max_length,
    max_prompt_length=512,
    beta=0.1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    report_to="wandb",
    output_dir="./results/",
)

trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    processing_class=tokenizer,
)
trainer.train()
trainer.save_model(new_model)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:274: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/990 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
25,7.230400,1.987780,4.595300,2.176000,1.088000,-0.158940,-0.180446,0.600000,0.021506,-1.804455,-1.589400,3.595057,3.123071,1.927990,-0.597908,0.255791
50,6.863800,1.941627,4.588300,2.179000,1.090000,-0.154829,-0.174185,0.600000,0.019356,-1.741850,-1.548287,3.266427,2.697751,1.880938,-0.606888,0.233354
75,6.709500,1.914187,4.594600,2.176000,1.088000,-0.152584,-0.170910,0.600000,0.018326,-1.709097,-1.525839,3.113975,2.496592,1.853035,-0.611520,0.222726
100,8.690900,1.899081,4.606700,2.171000,1.085000,-0.151469,-0.169497,0.600000,0.018029,-1.694974,-1.514688,3.083155,2.444144,1.837798,-0.612838,0.219970


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [17]:
dataset.column_names

{'train': ['source', 'chosen', 'rejected', 'prompt', 'question'],
 'test': ['source', 'chosen', 'rejected', 'prompt', 'question']}

In [18]:
# Save the final model
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")

('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/tokenizer.json')

In [19]:
# Flush memory
del trainer, model
gc.collect()
gc.collect()
torch.cuda.empty_cache()

#

In [36]:
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)
fp16_model = AutoModelForCausalLM.from_pretrained(
     base_model,
     low_cpu_mem_usage=True,
     return_dict=True,
     torch_dtype=torch.float16,
     device_map="auto",
 )
fp16_model, tokenizer = setup_chat_format(fp16_model, tokenizer)

# # Merge adapter with base model
model = PeftModel.from_pretrained(fp16_model, new_model)
model = model.merge_and_unload()

In [31]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("./OrpoLlama-3.2-1B-V1/")
tokenizer = AutoTokenizer.from_pretrained("./OrpoLlama-3.2-1B-V1/")

# model.push_to_hub("your-username/your-model-name")
# tokenizer.push_to_hub("your-username/your-model-name")


Loading adapter weights from ./OrpoLlama-3.2-1B-V1/ led to unexpected keys not found in the model: lm_head.weight, model.embed_tokens.weight, model.layers.0.mlp.down_proj.lora_A.default.weight, model.layers.0.mlp.down_proj.lora_B.default.weight, model.layers.0.mlp.gate_proj.lora_A.default.weight, model.layers.0.mlp.gate_proj.lora_B.default.weight, model.layers.0.mlp.up_proj.lora_A.default.weight, model.layers.0.mlp.up_proj.lora_B.default.weight, model.layers.0.self_attn.k_proj.lora_A.default.weight, model.layers.0.self_attn.k_proj.lora_B.default.weight, model.layers.0.self_attn.o_proj.lora_A.default.weight, model.layers.0.self_attn.o_proj.lora_B.default.weight, model.layers.0.self_attn.q_proj.lora_A.default.weight, model.layers.0.self_attn.q_proj.lora_B.default.weight, model.layers.0.self_attn.v_proj.lora_A.default.weight, model.layers.0.self_attn.v_proj.lora_B.default.weight, model.layers.1.mlp.down_proj.lora_A.default.weight, model.layers.1.mlp.down_proj.lora_B.default.weight, model.

In [38]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128258, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [39]:
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/bhuvana-ak7/OrpoLlama-3.2-1B-V1/commit/92de49d8b237401e639c6485e431a0fc72216b95', commit_message='Upload tokenizer', commit_description='', oid='92de49d8b237401e639c6485e431a0fc72216b95', pr_url=None, pr_revision=None, pr_num=None)

# Model Evaluation using EleutherAI/lm-evaluation-harness
## Install Eleuther Evaluation Harness



In [33]:

%%bash
git clone https://github.com/EleutherAI/lm-evaluation-harness
cd lm-evaluation-harness
pip install -e .

Obtaining file:///content/lm-evaluation-harness
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 

Cloning into 'lm-evaluation-harness'...


## Evaluate the LLM using Eleuther Evaluation Harness


hellaswag

In [41]:
%%bash
lm_eval --model hf \
    --model_args pretrained=bhuvana-ak7/OrpoLlama-3.2-1B-V1,dtype="float" \
    --tasks hellaswag \
    --device cuda \
    --batch_size auto:4 \
    --output_path hellaswag_test

Passed argument batch_size = auto:4.0. Detecting largest batch size
Determined largest batch size: 32
Passed argument batch_size = auto:4.0. Detecting largest batch size
Determined largest batch size: 32
Passed argument batch_size = auto:4.0. Detecting largest batch size
Determined largest batch size: 64
hf (pretrained=bhuvana-ak7/OrpoLlama-3.2-1B-V1,dtype=float), gen_kwargs: (None), limit: None, num_fewshot: None, batch_size: auto:4 (32,32,64,64,64)
|  Tasks  |Version|Filter|n-shot| Metric |   |Value |   |Stderr|
|---------|------:|------|-----:|--------|---|-----:|---|-----:|
|hellaswag|      1|none  |     0|acc     |↑  |0.4772|±  |0.0050|
|         |       |none  |     0|acc_norm|↑  |0.6366|±  |0.0048|



2024-10-28 02:18:01.526013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 02:18:01.547895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 02:18:01.554445: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 02:18:02.768097: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-10-28:02:18:04,723 INFO     [__main__.py:279] Verbosity set to INFO
2024-10-28:02:18:13,767 INFO     [__init__.py:459] The tag 'arc_ca' is already registered as a group, this tag will not be registered. This may affect tasks you want to

ifeval

In [42]:
! pip install -U langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=d961e89b5e6bc870df0b532ed97231c84d5cb490112413603e21c4b44d93a7f6
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [43]:
!pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.3 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [54]:
!pip install -U git+https://github.com/felipemaiapolo/tinyBenchmarks.git

  Cloning https://github.com/felipemaiapolo/tinyBenchmarks.git to /tmp/pip-req-build-bw46rjzc
  Running command git clone --filter=blob:none --quiet https://github.com/felipemaiapolo/tinyBenchmarks.git /tmp/pip-req-build-bw46rjzc
  Resolved https://github.com/felipemaiapolo/tinyBenchmarks.git to commit 9c7e20302301ad531bfdfd9a7288e6e916bf22e9
  Preparing metadata (setup.py) ... done
  Created wheel for tinyBenchmarks: filename=tinyBenchmarks-1.0.0-py3-none-any.whl size=6234 sha256=526c4996f8750cde373f1918c72acd5c676a0277ed3feff11211505ebf09103a
  Stored in directory: /tmp/pip-ephem-wheel-cache-mcuvvelj/wheels/2b/f0/1c/6b4b2371773774c7fb707fbbde1e52585f1d3963097ecf85f4
Successfully built tinyBenchmarks


In [55]:
%%bash
lm_eval --model hf \
    --model_args pretrained=bhuvana-ak7/OrpoLlama-3.2-1B-V1,dtype="float" \
    --tasks tinyMMLU \
    --device cuda \
    --batch_size auto:4 \
    --output_path tinyMMLU_test

Passed argument batch_size = auto:4.0. Detecting largest batch size
Determined largest batch size: 1
Passed argument batch_size = auto:4.0. Detecting largest batch size
Determined largest batch size: 4
Passed argument batch_size = auto:4.0. Detecting largest batch size
Determined largest batch size: 4
Passed argument batch_size = auto:4.0. Detecting largest batch size
Determined largest batch size: 8
hf (pretrained=bhuvana-ak7/OrpoLlama-3.2-1B-V1,dtype=float), gen_kwargs: (None), limit: None, num_fewshot: None, batch_size: auto:4 (1,4,4,8)
| Tasks  |Version|Filter|n-shot| Metric |   |Value |   |Stderr|
|--------|------:|------|-----:|--------|---|-----:|---|------|
|tinyMMLU|      0|none  |     0|acc_norm|↑  |0.4306|±  |   N/A|



2024-10-28 03:31:22.235552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 03:31:22.256752: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 03:31:22.263219: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 03:31:23.438750: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-10-28:03:31:25,293 INFO     [__main__.py:279] Verbosity set to INFO
2024-10-28:03:31:33,852 INFO     [__init__.py:459] The tag 'arc_ca' is already registered as a group, this tag will not be registered. This may affect tasks you want to

In [56]:
%%bash
lm_eval --model hf \
    --model_args pretrained=bhuvana-ak7/OrpoLlama-3.2-1B-V1,dtype="float" \
    --tasks eq_bench \
    --device cuda \
    --batch_size auto:4 \
    --output_path eq_bench_test

Passed argument batch_size = auto. Detecting largest batch size
Determined Largest batch size: 1
hf (pretrained=bhuvana-ak7/OrpoLlama-3.2-1B-V1,dtype=float), gen_kwargs: (None), limit: None, num_fewshot: None, batch_size: auto:4
| Tasks  |Version|Filter|n-shot|     Metric      |   | Value  |   |Stderr|
|--------|------:|------|-----:|-----------------|---|-------:|---|-----:|
|eq_bench|    2.1|none  |     0|eqbench          |↑  |-12.9709|±  |2.9658|
|        |       |none  |     0|percent_parseable|↑  | 92.9825|±  |1.9592|



2024-10-28 03:35:57.263089: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 03:35:57.284258: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 03:35:57.290655: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 03:35:58.454036: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-10-28:03:36:00,302 INFO     [__main__.py:279] Verbosity set to INFO
2024-10-28:03:36:08,923 INFO     [__init__.py:459] The tag 'arc_ca' is already registered as a group, this tag will not be registered. This may affect tasks you want to

In [36]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [50]:
try:

    # Optional: Test the merged model
    # tokenizer = AutoTokenizer.from_pretrained(new_model)
    test_input = "Who are you ..."
    inputs = tokenizer(test_input, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=50,
            num_return_sequences=1,
            temperature=0.7
        )

    print("\nTest output:")
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

except Exception as e:
    print(f"An error occurred during model merging: {str(e)}")


Test output:
Who are you... really?
We're not exactly sure what this is, but it's a pretty cool way to find out who you are.
So, take a look at this guy. Now, take a look at the girl on the right


In [49]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [

    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="bhuvana-ak7/OrpoLlama-3.2-1B-V1",device="cuda",torch_dtype="auto")
print(pipe(messages, max_new_tokens=128)[0]['generated_text'][-1])  # Print the assistant's response


Device set to use cuda


ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [43]:
!pip install accelerate bitsandbytes
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, logging
import os

# Suppress warnings for cleaner output
logging.set_verbosity(logging.ERROR)

# Load the model and tokenizer using 8-bit quantization to reduce memory usage
model = AutoModelForCausalLM.from_pretrained("bhuvana-ak7/OrpoLlama-3.2-1B", load_in_8bit=True, device_map="cpu")
tokenizer = AutoTokenizer.from_pretrained("bhuvana-ak7/OrpoLlama-3.2-1B")

messages = [
    {"role": "user", "content": "Who are you?"},
]

# Prepare the input for the model
prompt = messages[0]["content"]
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# Custom generate function with safeguards
def generate_with_safeguards(input_ids, **kwargs):
    with torch.no_grad():
        for _ in range(5):  # Try generating up to 5 times
            try:
                outputs = model.generate(input_ids, **kwargs)
                return outputs  # Successful generation
            except RuntimeError as e:
                if "probability tensor contains" in str(e):
                    print("Encountered numerical instability, retrying with adjusted parameters...")
                    kwargs["temperature"] *= 0.9  # Reduce temperature
                    kwargs["top_k"] = int(kwargs["top_k"] * 0.9)  # Reduce top_k
                    if kwargs["temperature"] < 0.1 or kwargs["top_k"] < 10:
                        raise e  # Give up after excessive adjustments
                else:
                    raise e  # Re-raise other errors
        raise RuntimeError("Failed to generate text after multiple attempts.")  # Give up

# Generate text using the custom function
try:
    outputs = generate_with_safeguards(
        input_ids,
        max_new_tokens=50,
        num_return_sequences=1,
        temperature=0.7,
        top_k=50,
        repetition_penalty=1.2
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)
except RuntimeError as e:
    print(f"Error during generation: {e}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
